In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import os
from prettytable import PrettyTable
from pylab import *
from scipy.stats import wasserstein_distance
import random
import csv
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
input_shape

46

### Move Working Directory

In [4]:
# MaryClare's
#os.chdir('/Users/maryclaremartin/Documents/jup/ExtraSensory')

# Josh's
#os.chdir("D:/Research_Projects/REU2021-human-context-recognition/UCI_data")

# Harrison's
os.chdir("/Users/hkimr/Desktop/WPI Github/REU2021-human-context-recognition/UCI_data")

#os.chdir('../')
softmax = nn.Softmax(dim = 1)

### Load & Scale Data

In [5]:
##load and scale data
#returns scaled data (X) and labels (Y)
#label: array, list of activities to use (represented by integers)
#users: array, list of users whose data is to be used (represented by integers)

def start_data(label, users, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names

    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']

    y_train_subject = pd.read_csv('data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    GAN_data = GAN_data[~((GAN_data['Activity'] == 4) & (GAN_data['Subject'] == 27))] #CHANGE THIS
    
    X = GAN_data.iloc[:, :-2].values
    y = GAN_data[['Activity']].values
    
    return X, y

### The Generator

In [6]:
##### defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim).to(device)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

##calculates generator loss
#gen: generator
#disc: discriminator
#criterion1: loss function1
#criterion2: loss function2
#batch_size: batch size
#z_dim: number of dimensions in the latent space
def get_gen_loss(gen, disc, act, usr, criterion1, criterion2, criterion3, batch_size, z_dim, activities, users):
    latent_vectors = get_noise(batch_size, z_dim)
    act_vectors = get_act_matrix(batch_size, activities)
    usr_vectors = get_usr_matrix(batch_size, users)
    
    to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
    fake_features = gen(to_gen)
    
    disc.eval()
    pred_disc = disc(fake_features)
    disc.train()
    
    pred_act = act(fake_features) 
    pred_usr = usr(fake_features)
    
    d_loss = criterion1(pred_disc, torch.ones_like(pred_disc))
    act_loss = criterion2(pred_act, act_vectors[0])
    usr_loss = criterion2(pred_usr, usr_vectors[0])       
    
    gen_loss = d_loss + act_loss + usr_loss
    return gen_loss
    
def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long().to(device), torch.Tensor(one_hot).to(device)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long().to(device), torch.Tensor(one_hot).to(device)

### The Discriminator

In [7]:
def discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

#defines discriminator class
class Discriminator(nn.Module):
    def __init__(self, feature_dim = input_shape, hidden_dim = 16):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            discriminator_block(feature_dim, 25),
            discriminator_block(25, 10),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )
    def forward(self, feature_vector):
        return self.disc(feature_vector)
    
def get_disc_loss(gen, disc, criterion, real_features, batch_size, z_dim, a_dim, u_dim):
    latent_vectors = get_noise(batch_size, z_dim)
    act_vectors = get_act_matrix(batch_size, a_dim)
    usr_vectors = get_usr_matrix(batch_size, u_dim)
    
    to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
    gen.eval()
    fake_features = gen(to_gen)
    gen.train()
    pred_fake = disc(fake_features.detach())
    
    ground_truth = torch.zeros_like(pred_fake)
    loss_fake = criterion(pred_fake, ground_truth)
    
    pred_real = disc(real_features)
    ground_truth = torch.ones_like(pred_real)
    loss_real = criterion(pred_real, ground_truth)
    
    disc_loss = (loss_fake + loss_real) / 2
    return disc_loss

### User Classifier

In [8]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class User_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(User_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 30),
            classifier_block(30, 20),
            classifier_block(20, 20),
            classifier_block(20, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)

### Activity Classifier

In [9]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Activity_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Activity_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 30),
            classifier_block(30, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)

### Calculate Performance Statistics

In [10]:
##calculates performance statistics for each epoch of training
#gen: generator
#disc: discriminator
#b_size: batch size
#z_dim: number of dimensions of the latent space
##returns accuracy, precision, recall, fpR, and f1 score
def performance_stats(gen, disc, b_size, z_dim, a_dim, u_dim, batch = None):
    tp = 0 #true positive
    fp = 0 #false positive
    tn = 0 #true negative
    fn = 0 #false negative

    with torch.no_grad():
        y_hat_fake = torch.Tensor([0])
        y_hat_real = torch.Tensor([0])
        
        if batch is None:
            latent_vectors = get_noise(b_size, z_dim)
            act_vectors = get_act_matrix(b_size, a_dim)
            usr_vectors = get_usr_matrix(b_size, u_dim)
            to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
            gen.eval()
            fake_features = gen(to_gen)
            gen.train()
            disc.eval()
            y_hat_fake = torch.round(disc(fake_features))
            disc.train()
        else:
            latent_vectors = get_noise(b_size, z_dim)
            act_vectors = get_act_matrix(b_size, a_dim)
            usr_vectors = get_usr_matrix(b_size, u_dim)
            to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
            gen.eval()
            fake_features = gen(to_gen)
            gen.train()
            disc.eval()
            y_hat_fake = torch.round(disc(fake_features))
            y_hat_real = torch.round(disc(batch))
            disc.train()
         
        fpR = torch.mean(y_hat_fake)
        recall = torch.mean(y_hat_real)
        class_acc = ((len(y_hat_fake) - torch.sum(y_hat_fake)) + torch.sum(y_hat_real)) / (2*b_size)
        #print(f'Classification Accuracy: {class_acc:.2f}'
        #print(f'Recall: {recall:.2f}') #What percent of the true positives were identified
        return class_acc, recall, fpR
      

def performance_stats_class(gen, a_class, u_class, batch_size, z_dim, a_dim, u_dim):
    tp = 0 #true positive
    fp = 0 #false positive
    tn = 0 #true negative
    fn = 0 #false negative

    with torch.no_grad():
        latent_vectors = get_noise(batch_size, z_dim)
        act_vectors = get_act_matrix(batch_size, a_dim)
        usr_vectors = get_usr_matrix(batch_size, u_dim)
    
        to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
        gen.eval()
        fake_features = gen(to_gen)
        gen.train()
    
        _, pred_a_class = torch.max(softmax(a_class(fake_features)), dim = 1)
        _, pred_u_class = torch.max(softmax(u_class(fake_features)), dim = 1)
        
        return torch.eq(act_vectors[0], pred_a_class).sum()/batch_size, torch.eq(usr_vectors[0], pred_u_class).sum()/batch_size

### Initialize Training Environment

In [11]:
###initalize parameters that depend on training loop parameters
#X: acceleration data
#y: labels associated with X data (fake or real)
#z_dim: number of dimensions to the latent space
#disc_lr: discriminator learning rate
#gen_lr: generator learning rate
#DISCRIMINATOR: 1 to indicate if discriminator is training
#batch_size: batch size
#disc: initialized discrimiantor

def initialize_params(X, y, z_dim, a_dim, u_dim, disc_lr, gen_lr, DISCRIMINATOR, batch_size, disc):
    #initialize generator
    gen = Generator(z_dim + a_dim + u_dim).to(device)
    #indicate that discriminator is training
    to_train = DISCRIMINATOR
    #create training features
    train_features = torch.tensor(X)
    #create training labels
    train_labels = torch.tensor(y)
    #concatenate to create training data
    train_data = torch.utils.data.TensorDataset(train_features, train_labels)
    #create data loader for training data
    train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
    #initialize generator and discriminator optimizers
    opt_disc = optim.Adam(disc.parameters(), lr = disc_lr)
    opt_gen = optim.Adam(gen.parameters(), lr = gen_lr)
    
    return gen, to_train, train_features, train_labels, train_data, train_loader, opt_disc, opt_gen   

# Save / Load Models

In [12]:
# Change path and name of the Generator and Discriminator accordingly
def save_model(gen, disc, model_name):
    torch.save(gen.state_dict(), f"saved_models/{model_name}_gen.param")
    torch.save(disc.state_dict(), f"saved_models/{model_name}_disc.param")
    
def load_model(model, model_name):
    model.load_state_dict(torch.load(f'saved_models/{model_name}'))

# Training

In [13]:
######Training loop to train GAN

#Parameters to specifiy: 
    #X: starting accelerometer data
    #y: starting labels for X data (fake or real)
    
#Set parameters (do not change)
    #criterion: loss function (BCE)
    #dig: number of significant digits for printing (5)
    #feature_dim: Number of dimensions of output from generator (26)
    #GENERATOR: set generator to zero for training
    #DISCRIMINATOR: set discriminator to one for training
    #train_string: starting machine to train (DISC)
    #disc: initalize discriminator
    #rel_epochs: Epochs passed since last switch (constant training) (0)
    #rows: initialization of array to save data of each epoch to CSV file ([])
    #heading: array of column headings for table (["Epoch", "Machine Training", "Discriminator Loss", 
                    #"Generator Loss", "FPR", "Recall", "Median Wasserstein", "Mean Wasserstein"])
    #table: intialize a table as a pretty table to save epoch data
    #switch_count: number of switches in dynamic training (0)
    
#Set parameters (can change):
    #z_dim: number of dimensions of latent vector (100)
    #gen_lr: generator learning rate (.001)
    #disc_lr: discriminator learning rate (.001) (shoud be equal to gen_lr)
    #batch_size: batch size (75)
    #print_batches: Show model performance per batch (False)
    #n_epochs: number of epochs to train (100)
    #constant_train_flag: (False)
        #Set to true to train based on constant # of epochs per machine 
        #Set to false to train dynamically based on machine performance
        
    #Constant training approach:
        #disc_epochs: Number of consecutive epochs to train discriminator before epoch threshold (5)
        #gen_epochs: Number of consecutive epochs to train generator before epoch threshold (2)
        #epoch_threshold: Epoch number to change training epoch ratio (50)
        #disc_epochs_change: New number of consecutive epochs to train discriminator after epoch threshold is exceeded (1)
        #gen_epochs_change: New number of consecutive epochs to train generator after epoch threshold is exceeded (50)
    
    #Dynamic training approach:                        
        #static_threshold: Epoch number to change from static ratio to dynamic (18)
        #static_disc_epochs: Number of consecutive epochs to train discriminator before epoch threshold (4)
        #static_gen_epochs: Number of consecutive epochs to train generator before epoch threshold (2)
        #pull_threshold: Accuracy threshold for switching machine training when the generator is no longer competitive (0.4)
        #push_threshold: Accuracy threshold for switching machine training when the discriminator is no longer competitive (0.6)
        #recall_threshold: threshold for recall to switch machine training when discriminator is training well
        #switch_flag: indicates if we should switch our training machine (False)
        
def training_loop(X, y, act, usr, criterion1 = nn.BCELoss(), criterion2 = nn.CrossEntropyLoss(), criterion3 = nn.CrossEntropyLoss(), gan_id = "Mod Test Gan", dig = 5, feature_dim = input_shape, 
                  GENERATOR = 0, DISCRIMINATOR = 1, train_string = "DISC", disc = Discriminator(), z_dim = 100, a_dim = 3, u_dim = 3, 
                  gen_lr =  0.001, disc_lr = 0.001, batch_size = 100, constant_train_flag = False, disc_epochs = 5,
                  gen_epochs = 2, epoch_threshold = 50, disc_epochs_change = 5, gen_epochs_change = 2, rel_epochs = 0,
                 static_threshold = 28, static_disc_epochs = 5, static_gen_epochs = 2, pull_threshold = 0.3,
                 push_threshold = 0.7, recall_threshold = 0.75, n_epochs = 1000, rows = [],
                 heading = ["Epoch", "Training", "Discriminator Loss", "Generator Loss", "D_Accuracy", "D_fpR", "D_Recall", "A_fpR", "U_fpR"],
                 table = PrettyTable(), switch_flag = False, switch_count = 0, last_real_features = []):
    
    disc.to(device)
    #returns generator, sets discriminator training, creates training tensor, loads data, and initializes optimizers
    gen, to_train, train_features, train_labels, train_data, train_loader, opt_disc, opt_gen = initialize_params(X, y, z_dim, a_dim, u_dim, disc_lr, gen_lr, DISCRIMINATOR, batch_size, disc)

    #set pretty table field names
    table.field_names = heading
    
    #visualize_gen(X, gen, z_dim, a_dim, u_dim)
    gen_epochs = 0
    
    last_D_loss = -1.0
    last_G_loss = -1.0
    
    mean_mean = []
    mean_median = []
    
    for epoch in range(n_epochs):  
        
#         pull_threshold = new_threshold(epoch)
        
#         if epoch < 300:
#             recall_threshold = 0.99
#         else:
#             recall_threshold = 0.75
            
        if constant_train_flag:
            if to_train == DISCRIMINATOR and rel_epochs >= disc_epochs:
                rel_epochs = 0
                to_train = GENERATOR
                train_string = "GEN"

            elif to_train == GENERATOR and rel_epochs >= gen_epochs:
                rel_epochs = 0
                to_train = DISCRIMINATOR
                train_string = "DISC"

            # Change epoch ratio after intial 'leveling out'
            if epoch == epoch_threshold:
                rel_epochs = 0
                to_train = GENERATOR
                train_string = "GEN"

                old_ratio = gen_epochs / disc_epochs
                gen_epochs = gen_epochs_change
                disc_epochs = disc_epochs_change
                new_ratio = gen_epochs / disc_epochs
                print(f'\n\nTraining ratio of G/D switched from {old_ratio:.{dig}f} to {new_ratio:.{dig}f}\n\n')
        else:
            if epoch < static_threshold:
                if to_train == DISCRIMINATOR and rel_epochs >= static_disc_epochs:
                    rel_epochs = 0
                    to_train = GENERATOR
                    train_string = "GEN"

                elif to_train == GENERATOR and rel_epochs >= static_gen_epochs:
                    rel_epochs = 0
                    to_train = DISCRIMINATOR
                    train_string = "DISC"

            else:
                if not switch_flag:
                    print("\nSwitching to Dynamic Training\n")
                    switch_flag = True
                    to_train = DISCRIMINATOR
                    train_string = "DISC"
                if to_train == DISCRIMINATOR and fpR <= pull_threshold and R >= recall_threshold:
                    to_train = GENERATOR
                    train_string = "GEN"
                    print("\nPull Generator\n")
                    switch_count += 1
                if to_train == GENERATOR and fpR >= push_threshold:
                    to_train = DISCRIMINATOR
                    train_string = "DISC"
                    print("\nPush Generator\n")
                    switch_count += 1
                    
        print(f'Epoch[{epoch + 1}/{n_epochs}] Train: {train_string} ', end ='')
        
        for batch_idx, (real_features, _) in enumerate(train_loader):
            #Size of this current batch
            batch_size = len(real_features)
            #Send batch to GPU
            real_features = real_features.float().to(device)
            
            if to_train == DISCRIMINATOR:
                ### Training Discriminator
                #visualize_real_batch(real_features.float())
                opt_disc.zero_grad()
                disc_loss = get_disc_loss(gen, disc, criterion1, real_features, batch_size, z_dim, a_dim, u_dim)
                disc_loss.backward(retain_graph = True)
                opt_disc.step()
                last_D_loss = disc_loss.item()
                # If generator has yet to ever train, run intial evaluation
                if last_G_loss == -1.0:
                    last_G_loss = get_gen_loss(gen, disc, act, usr, criterion1, criterion2, criterion3, batch_size, z_dim, a_dim, u_dim)
                
            else:
                ### Training Generator
                opt_gen.zero_grad()
                gen_loss = get_gen_loss(gen, disc, act, usr, criterion1, criterion2, criterion3, batch_size, z_dim, a_dim, u_dim)
                gen_loss.backward()
                opt_gen.step()
                last_G_loss = gen_loss.item()
                # If discriminator has yet to ever train, run intial evaluation
                if last_D_loss == -1.0:
                    last_D_loss = get_disc_loss(gen, disc, criterion1, real_features, batch_size, z_dim, a_dim, u_dim)
                
            if batch_idx == (len(train_loader) - 1):
                acc, R, fpR = performance_stats(gen, disc, batch_size, z_dim, a_dim, u_dim, batch = real_features)
                A_fpR, U_fpR = performance_stats_class(gen, act, usr, batch_size, z_dim, a_dim, u_dim)
                #w_dist = all_Wasserstein_dists(gen, z_dim, feature_dim, real_features.float())
                #median_w_dist = torch.median(w_dist)
                #mean_w_dist = torch.mean(w_dist)
                
                #mean_mean.append(mean_w_dist)
                #mean_median.append(median_w_dist)
                
        mean_mean_w = torch.mean(torch.Tensor(mean_mean)) 
        mean_median_w = torch.mean(torch.Tensor(mean_median))
            
        ### Currently doesn't print Median/Mean Wasserstein --> Change if needed
        print(f'| LossD: {last_D_loss:.{dig}f}, LossG: {last_G_loss:.{dig}f} | Acc: {acc:.{dig}f} | fpR: {fpR:.{dig}f} | R: {R:.{dig}f} | A_fpR: {A_fpR:.{dig}f} | U_fpR: {U_fpR:.{dig}f}')
        row_to_add = [f"{epoch + 1}", train_string, f"{last_D_loss:.{dig}f}", f"{last_G_loss:.{dig}f}", f"{acc:.{dig}f}", f"{fpR:.{dig}f}", f"{R:.{dig}f}", f"{A_fpR:.{dig}f}", f"{U_fpR:.{dig}f}"]
        table.add_row(row_to_add)
        rows.append(row_to_add)
                
        if to_train == GENERATOR:
            gen_epochs += 1
            
        mean_mean.clear()
        mean_median.clear()
        rel_epochs += 1
        
#         if epoch > 15000 and fpR < 0.1 and R < 0.65:
#             break
        
        
    print("\n\nTraining Session Finished")
    print(f"Encountered {switch_count} non-trivial training swaps")
    percent = gen_epochs / n_epochs
    print(f"Trained Generator {gen_epochs} out of {n_epochs} ({percent:.3f})")
    f = open("model_outputs/" + gan_id + ".txt", "w")
    f.write(table.get_string())
    f.close()
    print("Model Results Sucessfully Saved to \"model_outputs/" + gan_id + ".txt\"")

    with open("model_outputs/" + gan_id + ".csv", "w") as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 
        # writing the fields 
        csvwriter.writerow(heading)
        # writing the data rows 
        csvwriter.writerows(rows)
    print("Model Results Sucessfully Saved to \"model_outputs/" + gan_id + ".csv\"")
    save_model(gen, disc, gan_id)
    print("Generator's Trained Parameters Sucessfully Saved to \"saved_models/" + gan_id + "_gen.param\"")
    print("Discriminators's Trained Parameters Sucessfully Saved to \"saved_models/" + gan_id + "_disc.param\"")
    model_output = pd.read_csv("model_outputs/" + gan_id + ".csv")
    #visualize_gen(X, gen, z_dim)
    # Change path and name of the Generator and Discriminator accordingly
    save_model(gen, disc, gan_id)
    
    return model_output

# Run Training

In [14]:
#X, y = start_data("aggregated_data/aggregated_data.csv", "label:SITTING")
#X, y = start_data("raw_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv", "label:SITTING" ) 
X, y = start_data([1, 3, 4], [23, 25, 27], sub_features, act_features)

#Initialize Classifier Architectures
activity_classifier = Activity_Classifier()
user_classifier = User_Classifier()

#Freeze Them
activity_classifier.eval()
user_classifier.eval()
#Send to GPU
activity_classifier.to(device)
user_classifier.to(device)

#Load with pre-trained parameters
activity_classifier.load_state_dict(torch.load('saved_models/UCI 3 Label Classifier Group 5'))
user_classifier.load_state_dict(torch.load("saved_models/UCI 3 User Classifier Group 5"))

#Train
model_output = training_loop(X,y, activity_classifier, user_classifier, gan_id="U2A2 Excluded Group 5", batch_size = len(X), gen_lr=.0001, disc_lr =.0001, n_epochs=20000, dig=5, constant_train_flag=False)

Epoch[1/20000] Train: DISC | LossD: 0.70446, LossG: 50.96322 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000 | A_fpR: 0.34864 | U_fpR: 0.31733
Epoch[2/20000] Train: DISC | LossD: 0.70481, LossG: 50.96322 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000 | A_fpR: 0.32985 | U_fpR: 0.31733
Epoch[3/20000] Train: DISC | LossD: 0.70439, LossG: 50.96322 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000 | A_fpR: 0.34238 | U_fpR: 0.35073
Epoch[4/20000] Train: DISC | LossD: 0.70224, LossG: 50.96322 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000 | A_fpR: 0.32985 | U_fpR: 0.32985
Epoch[5/20000] Train: DISC | LossD: 0.70361, LossG: 50.96322 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000 | A_fpR: 0.30898 | U_fpR: 0.31942
Epoch[6/20000] Train: GEN | LossD: 0.70361, LossG: 48.44483 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000 | A_fpR: 0.33612 | U_fpR: 0.35491
Epoch[7/20000] Train: GEN | LossD: 0.70361, LossG: 45.04800 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000 | A_fpR: 0.35282 | U_fpR: 0.30898
Epoch[8/20000] Train: DISC | LossD: 

Epoch[68/20000] Train: DISC | LossD: 0.67487, LossG: 42.97685 | Acc: 0.63779 | fpR: 0.00000 | R: 0.27557 | A_fpR: 0.30480 | U_fpR: 0.31942
Epoch[69/20000] Train: DISC | LossD: 0.67298, LossG: 42.97685 | Acc: 0.63883 | fpR: 0.00000 | R: 0.27766 | A_fpR: 0.30063 | U_fpR: 0.33403
Epoch[70/20000] Train: DISC | LossD: 0.67478, LossG: 42.97685 | Acc: 0.63883 | fpR: 0.00000 | R: 0.27766 | A_fpR: 0.29436 | U_fpR: 0.31733
Epoch[71/20000] Train: DISC | LossD: 0.67301, LossG: 42.97685 | Acc: 0.63883 | fpR: 0.00000 | R: 0.27766 | A_fpR: 0.35282 | U_fpR: 0.32359
Epoch[72/20000] Train: DISC | LossD: 0.67213, LossG: 42.97685 | Acc: 0.63883 | fpR: 0.00000 | R: 0.27766 | A_fpR: 0.35491 | U_fpR: 0.31733
Epoch[73/20000] Train: DISC | LossD: 0.67183, LossG: 42.97685 | Acc: 0.63883 | fpR: 0.00000 | R: 0.27766 | A_fpR: 0.30063 | U_fpR: 0.34029
Epoch[74/20000] Train: DISC | LossD: 0.67219, LossG: 42.97685 | Acc: 0.63883 | fpR: 0.00000 | R: 0.27766 | A_fpR: 0.35699 | U_fpR: 0.30480
Epoch[75/20000] Train: DISC

Epoch[136/20000] Train: GEN | LossD: 0.63744, LossG: 37.65357 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.32985 | U_fpR: 0.33194
Epoch[137/20000] Train: GEN | LossD: 0.63744, LossG: 40.27133 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.36534 | U_fpR: 0.27349
Epoch[138/20000] Train: GEN | LossD: 0.63744, LossG: 37.04514 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.35282 | U_fpR: 0.34029
Epoch[139/20000] Train: GEN | LossD: 0.63744, LossG: 40.19189 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.34238 | U_fpR: 0.36117
Epoch[140/20000] Train: GEN | LossD: 0.63744, LossG: 36.80818 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.31315 | U_fpR: 0.34656
Epoch[141/20000] Train: GEN | LossD: 0.63744, LossG: 39.30898 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.38413 | U_fpR: 0.32985
Epoch[142/20000] Train: GEN | LossD: 0.63744, LossG: 35.48643 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.36743 | U_fpR: 0.32777
Epoch[143/20000] Train: GEN

Epoch[195/20000] Train: GEN | LossD: 0.63744, LossG: 23.80299 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.35908 | U_fpR: 0.37161
Epoch[196/20000] Train: GEN | LossD: 0.63744, LossG: 24.48755 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.35491 | U_fpR: 0.35908
Epoch[197/20000] Train: GEN | LossD: 0.63744, LossG: 24.19012 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.34447 | U_fpR: 0.34656
Epoch[198/20000] Train: GEN | LossD: 0.63744, LossG: 23.94464 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.34029 | U_fpR: 0.35282
Epoch[199/20000] Train: GEN | LossD: 0.63744, LossG: 26.26427 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.36117 | U_fpR: 0.39248
Epoch[200/20000] Train: GEN | LossD: 0.63744, LossG: 24.86258 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.39457 | U_fpR: 0.39666
Epoch[201/20000] Train: GEN | LossD: 0.63744, LossG: 24.52969 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.40501 | U_fpR: 0.36117
Epoch[202/20000] Train: GEN

Epoch[264/20000] Train: GEN | LossD: 0.63744, LossG: 18.61284 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.37787 | U_fpR: 0.34238
Epoch[265/20000] Train: GEN | LossD: 0.63744, LossG: 18.19893 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.35699 | U_fpR: 0.42589
Epoch[266/20000] Train: GEN | LossD: 0.63744, LossG: 19.21047 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.38205 | U_fpR: 0.36534
Epoch[267/20000] Train: GEN | LossD: 0.63744, LossG: 17.93537 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.35491 | U_fpR: 0.41336
Epoch[268/20000] Train: GEN | LossD: 0.63744, LossG: 16.49961 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.34864 | U_fpR: 0.35073
Epoch[269/20000] Train: GEN | LossD: 0.63744, LossG: 18.25215 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.41545 | U_fpR: 0.40710
Epoch[270/20000] Train: GEN | LossD: 0.63744, LossG: 17.20240 | Acc: 0.87683 | fpR: 0.00000 | R: 0.75365 | A_fpR: 0.40084 | U_fpR: 0.37578
Epoch[271/20000] Train: GEN

...

Epoch[19683/20000] Train: DISC | LossD: 0.65737, LossG: 0.62647 | Acc: 0.63570 | fpR: 0.29645 | R: 0.56785 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19684/20000] Train: DISC | LossD: 0.66279, LossG: 0.62647 | Acc: 0.63361 | fpR: 0.30271 | R: 0.56994 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19685/20000] Train: DISC | LossD: 0.66078, LossG: 0.62647 | Acc: 0.65240 | fpR: 0.27140 | R: 0.57620 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19686/20000] Train: DISC | LossD: 0.66271, LossG: 0.62647 | Acc: 0.63361 | fpR: 0.30898 | R: 0.57620 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19687/20000] Train: DISC | LossD: 0.66007, LossG: 0.62647 | Acc: 0.65449 | fpR: 0.27140 | R: 0.58038 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19688/20000] Train: DISC | LossD: 0.66000, LossG: 0.62647 | Acc: 0.63361 | fpR: 0.31524 | R: 0.58246 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19689/20000] Train: DISC | LossD: 0.66233, LossG: 0.62647 | Acc: 0.65344 | fpR: 0.27975 | R: 0.58664 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19690/2

Epoch[19756/20000] Train: DISC | LossD: 0.62827, LossG: 0.62647 | Acc: 0.71608 | fpR: 0.25470 | R: 0.68685 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19757/20000] Train: DISC | LossD: 0.62849, LossG: 0.62647 | Acc: 0.71712 | fpR: 0.25470 | R: 0.68894 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19758/20000] Train: DISC | LossD: 0.62207, LossG: 0.62647 | Acc: 0.71190 | fpR: 0.27140 | R: 0.69520 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19759/20000] Train: DISC | LossD: 0.62871, LossG: 0.62647 | Acc: 0.74008 | fpR: 0.21712 | R: 0.69729 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19760/20000] Train: DISC | LossD: 0.62565, LossG: 0.62647 | Acc: 0.72651 | fpR: 0.25052 | R: 0.70355 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19761/20000] Train: DISC | LossD: 0.62464, LossG: 0.62647 | Acc: 0.72965 | fpR: 0.24635 | R: 0.70564 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19762/20000] Train: DISC | LossD: 0.62380, LossG: 0.62647 | Acc: 0.72860 | fpR: 0.24843 | R: 0.70564 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19763/2

Epoch[19818/20000] Train: GEN | LossD: 0.61892, LossG: 0.64728 | Acc: 0.60960 | fpR: 0.53236 | R: 0.75157 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19819/20000] Train: GEN | LossD: 0.61892, LossG: 0.64460 | Acc: 0.59603 | fpR: 0.55950 | R: 0.75157 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19820/20000] Train: GEN | LossD: 0.61892, LossG: 0.64324 | Acc: 0.59290 | fpR: 0.56576 | R: 0.75157 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19821/20000] Train: GEN | LossD: 0.61892, LossG: 0.62724 | Acc: 0.56785 | fpR: 0.61587 | R: 0.75157 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19822/20000] Train: GEN | LossD: 0.61892, LossG: 0.64925 | Acc: 0.54802 | fpR: 0.65553 | R: 0.75157 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19823/20000] Train: GEN | LossD: 0.61892, LossG: 0.63685 | Acc: 0.55532 | fpR: 0.64092 | R: 0.75157 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19824/20000] Train: GEN | LossD: 0.61892, LossG: 0.62638 | Acc: 0.56054 | fpR: 0.63048 | R: 0.75157 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19825/20000] T

Epoch[19884/20000] Train: DISC | LossD: 0.65509, LossG: 0.61277 | Acc: 0.65762 | fpR: 0.22129 | R: 0.53653 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19885/20000] Train: DISC | LossD: 0.66597, LossG: 0.61277 | Acc: 0.63466 | fpR: 0.26722 | R: 0.53653 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19886/20000] Train: DISC | LossD: 0.66220, LossG: 0.61277 | Acc: 0.64927 | fpR: 0.23800 | R: 0.53653 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19887/20000] Train: DISC | LossD: 0.66416, LossG: 0.61277 | Acc: 0.64718 | fpR: 0.24426 | R: 0.53862 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19888/20000] Train: DISC | LossD: 0.65403, LossG: 0.61277 | Acc: 0.64927 | fpR: 0.24008 | R: 0.53862 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19889/20000] Train: DISC | LossD: 0.66340, LossG: 0.61277 | Acc: 0.65136 | fpR: 0.23591 | R: 0.53862 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19890/20000] Train: DISC | LossD: 0.65060, LossG: 0.61277 | Acc: 0.66597 | fpR: 0.21086 | R: 0.54280 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19891/2

Epoch[19955/20000] Train: DISC | LossD: 0.62502, LossG: 0.61277 | Acc: 0.71086 | fpR: 0.22547 | R: 0.64718 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19956/20000] Train: DISC | LossD: 0.61911, LossG: 0.61277 | Acc: 0.70877 | fpR: 0.23382 | R: 0.65136 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19957/20000] Train: DISC | LossD: 0.62116, LossG: 0.61277 | Acc: 0.73486 | fpR: 0.18163 | R: 0.65136 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19958/20000] Train: DISC | LossD: 0.62095, LossG: 0.61277 | Acc: 0.72338 | fpR: 0.20877 | R: 0.65553 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19959/20000] Train: DISC | LossD: 0.62223, LossG: 0.61277 | Acc: 0.73695 | fpR: 0.18163 | R: 0.65553 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19960/20000] Train: DISC | LossD: 0.61452, LossG: 0.61277 | Acc: 0.73069 | fpR: 0.19833 | R: 0.65971 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19961/20000] Train: DISC | LossD: 0.61476, LossG: 0.61277 | Acc: 0.72860 | fpR: 0.20877 | R: 0.66597 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19962/2